# 1. 開啟瀏覽器，進到FB首頁

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd


# 關閉通知
options = webdriver.ChromeOptions()
prefs = {
    'profile.default_content_setting_values':
        {
            'notifications': 2
        }
}
options.add_experimental_option('prefs', prefs)
options.add_argument("disable-infobars")

In [2]:
# 打啟動selenium 務必確認driver 檔案跟python 檔案要在同個資料夾中
# 在這逼下載: https://chromedriver.chromium.org/downloads
driver = webdriver.Chrome(options=options)
driver.get("https://www.facebook.com/")
time.sleep(4)

In [3]:
email = "your_email"
password = "your_password"

#輸入 email 
context = driver.find_element(By.ID, 'email')
context.send_keys(email)
time.sleep(0.5)

#輸入 password
context = driver.find_element(By.ID, 'pass')
context.send_keys(password)
time.sleep(0.5)

# 登入
commit = driver.find_element(By.NAME, 'login')
commit.click()
time.sleep(7)
print("已登入")

已登入


# 2. 進入台大交流版

## 2.1 手動選擇「最新貼文」

In [5]:
# 進入交流版
driver.get("https://www.facebook.com/groups/NTU.Head")

In [7]:
# 選擇最新貼文 (預設是熱門貼文)
# 點開選項
commit = driver.find_element(By.XPATH, "//div[@class='j83agx80']//span[@class='a8c37x1j ni8dbmo4 stjgntxs l9j0dhe7']")
commit.click()

# 三個選項，依序是「最新動態」、「最近貼文」、「最相關貼文」，選第二個
commit = driver.find_elements(By.XPATH, "//div[@class='bp9cbjyn j83agx80 btwxx1t3 buofh1pr i1fnvgqd hpfvmrgz']")
commit[1].click()

## 2.2 自動以「最新貼文」排序

In [4]:
# 進入交流版 (以最新貼文排序) (取代上兩格)
driver.get("https://www.facebook.com/groups/NTU.Head?sorting_setting=CHRONOLOGICAL")

# 3. 展開頁面

## 3.1 往下滑以顯示更多篇文章

In [5]:
# 往下滑
for i in range(0,5):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(5)
print("滑好了")

滑好了


## 3.2 點開「顯示更多」

In [15]:
# 點「顯示更多」 (要設兩層條件，因為貼文者那邊 其他6人也是div後面這個class) (要多執行幾次 直到數字不便)

commit = driver.find_elements(By.XPATH, "//div[@class='oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gpro0wi8 oo9gr5id lrazzd5p' and @role='button']")
print(len(commit))

for c in commit:
    try:
        c.click()
    except:
        print(c)

commit = driver.find_elements(By.XPATH, "//div[@class='oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gpro0wi8 oo9gr5id lrazzd5p' and @role='button']")
if len(commit) == 0:
    print("都點開了")
else:
    print("點開了")

1
<selenium.webdriver.remote.webelement.WebElement (session="8ba267b11457ce085af3da728e8a3ed3", element="906a71c7-7b8a-4c4c-8a74-522bb2ab9f11")>
點開了


## 3.3 將游標移至「時間」上，以方便後續獲取貼文連結

In [17]:
urls = driver.find_elements(By.XPATH, "//div[@class='du4w35lb k4urcfbm l9j0dhe7 sjgh65i0']//span[@class='tojvnm2t a6sixzi8 abs2jz4q a8s20v7p t1p8iaqh k5wvi7nf q3lfd5jv pk4s997a bipmatt0 cebpdrjk qowsmv63 owwhemhu dp1hu0rb dhp61c6y iyyx5f41']//a[@role='link']")
print(len(urls))

action = ActionChains(driver)
for url in urls:
    action.move_to_element(url).perform()
print("OK")

23
OK


# 4. 擷取網頁資料

## 4.1 擷取並解析 HTML

In [31]:
html = etree.HTML(driver.page_source)
soup = BeautifulSoup(driver.page_source, "html.parser")

## 4.2 選擇貼文區

In [32]:
# 貼文區是 role=feed，只會有一個
all_ = soup.find_all("div", role="feed")
print(len(all_))

1


## 4.3 獲取文章資訊

### 4.3.1 以一篇文章為單位，獲取「作者」、「時間」、「連結」、「內容」

In [33]:
# 一篇PO文
# div class_="du4w35lb k4urcfbm l9j0dhe7 sjgh65i0"
# 文字的部分
# div class_="ecm0bbzt hv4rvrfc ihqw7lf3 dati1w0a" (比"qzhwtbm6 knvmm38d"層級高)
# div class_="dati1w0a ihqw7lf3 hv4rvrfc ecm0bbzt"
# 每篇PO文的重要訊息們 (一般是3個)
# div class_="qzhwtbm6 knvmm38d"


all_2 = all_[0].find_all("div", class_="du4w35lb k4urcfbm l9j0dhe7 sjgh65i0")
print(len(all_2))

name_list = []
time_list = []
url_list = []
content_list = []

for i in range(len(all_2)):
    # inform 前2個固定是名字和時間，但後面數量不一定，依貼文形式而定
    inform = all_2[i].find_all("div", class_="qzhwtbm6 knvmm38d")
    print(i)
    #print("inform", len(inform))

    
    # 名字 name
    x = inform[0].find_all("h2")
    z = x[0].find("span").text
    print(z)
    name_list.append(z)

    # 時間 time
    m = inform[1].find_all("a", class_ = "oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw")
    n = m[0].find("span").text.strip("=")
    print(n)
    time_list.append(n)
    
    # 文章連結 (滑鼠游標移到上面之後 HTML裡才會有，不然都是#)
    url = m[0].get("href")
    print(url)
    url_list.append(url)
    
    # 內容 content
    try:
        j = inform[2].find_all("div", dir="auto")
        
        # 如果貼文的是一群人分享文章就會進到這邊
        if len(j) == 0:
            xyz = all_2[i].find_all("div", class_="dati1w0a ihqw7lf3 hv4rvrfc ecm0bbzt")
            w = xyz[0]
            print(w.text)
            content_list.append(w.text)
        
        # 一般文章走這邊
        else:
            txt = ""
            for k in j:
                print(k.text)
                txt += k.text     # 把不同行的內容結合在一起
            content_list.append(txt)

    except:
        # 如果貼文的是一群人單純 PO 文章就會進到這邊
        try:
            xyz = all_2[i].find_all("div", class_="dati1w0a ihqw7lf3 hv4rvrfc ecm0bbzt")
            w = xyz[0]
            print(w.text)
            content_list.append(w.text)
        
        except:
            print("Ooops")

    print("\n")

27
0
楚湘雨
45分鐘
https://www.facebook.com/groups/NTU.Head/posts/1948563748681744/?__cft__[0]=AZVi1ewFJCsQ0ndnaZ23xoV-yZvZQZdvlMWdS6RKlNi-p2GzfGt1M2ComAhyKd3Xon6usOi-Ox8FeuZttmwoN3vR6KnyB_VBsKC30mFUTcItaBMqBgP2mzCp0rUPH1RxKDoo6XtoosDdK6HO1mAExpPCoNv5LyOQu6TV15H133IpNOqNxWJ6PWG_T-4Y1_hfOis&__tn__=%2CO%2CP-R
討論意涵：運動時注意隨身物品
台大跟師大校園，最近出現一名竊賊，趁學生或民眾運動時，把隨身背包放在旁邊，偷走裡面的皮夾，還有學生把密碼寫在提款卡上，生活費全被盜領一空，警方調閱監視器，認出嫌犯是慣竊，過去當過台北市議員陳彥伯助理，但犯下許多竊盜案，早已被陳彥伯開除。這次供稱，找不到工作才會一直犯案。
學生在校園內的操場來來去去，身穿白色運動上衣的嫌犯，刻意在旁邊拉筋抬腿，但其實早就鎖定，旁邊的黑色包包，等到沒人後一屁股坐下，伸手進去偷取裡面的財物，現金和皮夾一次到手，有些學生把提款卡號碼，寫在卡片上，馬上被拿去超商盜領，事後一查，嫌犯過去還是前北市議員陳彥伯的助理，不過轄區警方人人都認識他，因為他已經從議員助理淪為慣竊。
嫌犯鎖定的都是大安區的校園，包括師大、台大都有他的犯案蹤跡，而且都是以體育場為主，警方清查，光是最近六次，他就盜領1萬1千塊，拿走現金2600元，受害者幾乎都是學生，從2014年起他還在一家設計公司工作，就4度潛入雇主住家和公司，得手40萬現金及鑽戒，女雇主相當生氣，當庭痛罵陳姓嫌犯拿鑰匙闖入，連祖母的手尾戒都被他偷走，被判刑9個月得易科罰金27萬。
但他沒有悔改，2017年又在健身房及圖書館，連偷4案被判刑10個月，今年更誇張，專門鎖定校園，這次警方清查，共犯下19個案子，判刑2年，過去擔任陳彥伯助理時，辦公室內也曾傳出，物品金錢不見，已將他開除 。
大安分局羅斯福路派出所所長楊進華：「持地院及地檢拘票及搜索票，至文山區萬盛街，陳嫌住處將其逮捕，並取回相關贓證物。」前議員助理淪為竊賊，供稱找不到工作才會一犯再犯，近年將目標，放在開放的大學校園，尤其是

### 4.3.2 列印出前3組資料

In [34]:
for i in range(3):
    print(name_list[i])
    print(time_list[i])
    print(url_list[i])
    print(content_list[i])
    print("\n")

楚湘雨
45分鐘
https://www.facebook.com/groups/NTU.Head/posts/1948563748681744/?__cft__[0]=AZVi1ewFJCsQ0ndnaZ23xoV-yZvZQZdvlMWdS6RKlNi-p2GzfGt1M2ComAhyKd3Xon6usOi-Ox8FeuZttmwoN3vR6KnyB_VBsKC30mFUTcItaBMqBgP2mzCp0rUPH1RxKDoo6XtoosDdK6HO1mAExpPCoNv5LyOQu6TV15H133IpNOqNxWJ6PWG_T-4Y1_hfOis&__tn__=%2CO%2CP-R
討論意涵：運動時注意隨身物品台大跟師大校園，最近出現一名竊賊，趁學生或民眾運動時，把隨身背包放在旁邊，偷走裡面的皮夾，還有學生把密碼寫在提款卡上，生活費全被盜領一空，警方調閱監視器，認出嫌犯是慣竊，過去當過台北市議員陳彥伯助理，但犯下許多竊盜案，早已被陳彥伯開除。這次供稱，找不到工作才會一直犯案。學生在校園內的操場來來去去，身穿白色運動上衣的嫌犯，刻意在旁邊拉筋抬腿，但其實早就鎖定，旁邊的黑色包包，等到沒人後一屁股坐下，伸手進去偷取裡面的財物，現金和皮夾一次到手，有些學生把提款卡號碼，寫在卡片上，馬上被拿去超商盜領，事後一查，嫌犯過去還是前北市議員陳彥伯的助理，不過轄區警方人人都認識他，因為他已經從議員助理淪為慣竊。嫌犯鎖定的都是大安區的校園，包括師大、台大都有他的犯案蹤跡，而且都是以體育場為主，警方清查，光是最近六次，他就盜領1萬1千塊，拿走現金2600元，受害者幾乎都是學生，從2014年起他還在一家設計公司工作，就4度潛入雇主住家和公司，得手40萬現金及鑽戒，女雇主相當生氣，當庭痛罵陳姓嫌犯拿鑰匙闖入，連祖母的手尾戒都被他偷走，被判刑9個月得易科罰金27萬。但他沒有悔改，2017年又在健身房及圖書館，連偷4案被判刑10個月，今年更誇張，專門鎖定校園，這次警方清查，共犯下19個案子，判刑2年，過去擔任陳彥伯助理時，辦公室內也曾傳出，物品金錢不見，已將他開除 。大安分局羅斯福路派出所所長楊進華：「持地院及地檢拘票及搜索票，至文山區萬盛街，陳嫌住處將其逮捕，並取回相關贓證物。」前議員助理淪為竊賊，供稱找不到工作才會一犯再犯，近年將目標，放在開放的大學校園，尤其是操場上，還是要提醒民

### 4.3.3 檢查前20組資料是否有關鍵字

In [35]:
keywords = ["免費", "便當", "剩下", "飲料"]
count = 0

# 依序添加「姓名」「時間」
dict = {"name":name_list, "time":time_list}

# 若「內容」含有關鍵字，則打勾，反之則留空 (不含分享的文章內容)
for keyword in keywords:
    has_keyword = []
    for i in range(len(content_list)):
        if keyword in content_list[i]:
            count += 1
            has_keyword.append("V")
        else:
            has_keyword.append(" ")
    dict[keyword] = has_keyword

# 把「內容」加在最後
dict["content"] = content_list
dict["url"] = url_list

print("在 " +str(len(content_list))+" 篇文中")
print("共發現 "+str(count)+" 篇文中含有關鍵字")
print("關鍵字:", keywords)

data = pd.DataFrame(dict)

在 27 篇文中
共發現 1 篇文中含有關鍵字
關鍵字: ['免費', '便當', '剩下', '飲料']


In [36]:
data

,name,time,免費,便當,剩下,飲料,content,url
0,楚湘雨,45分鐘,,,,,討論意涵：運動時注意隨身物品台大跟師大校園，最近出現一名竊賊，趁學生或民眾運動時，把隨身背包...,https://www.facebook.com/groups/NTU.Head/posts...
1,李崗,53分鐘,,,,,青年問禪師：“王力宏都離婚了，是否愛情本不存在？”禪師微微一笑，指著天空一隻鳥對青年道：“看...,https://www.facebook.com/groups/NTU.Head/posts...
2,梁祐祥,54分鐘,,,,,嗨 大家好我是 One-Forty 的 祐祥，想和大家分享 One-Forty 的徵才資訊2...,https://www.facebook.com/groups/NTU.Head/posts...
3,楊元嘉,1小時,,,,,【研討會食物發放】全部發完了謝謝大家大家好，社科院3樓有很多餐盒和夏威夷飯，請想要吃的大家趕...,https://www.facebook.com/groups/NTU.Head/posts...
4,簡韻宜,1 小時,,,,,在公館NET前面停腳踏車的地方撿到一副AirPods，16:20左右會送去駐警隊！希望他的主...,https://www.facebook.com/groups/NTU.Head/posts...
5,Chia-Yu Yang,2 小時,,,,,【拾獲學生證】電信工程學研究所的李同學週四晚上在普通外的樹洞裡看到你的學生證，因為插在那感覺...,https://www.facebook.com/groups/NTU.Head/posts...
6,楊健群,3 小時,,,,,〖台大心理系實驗室徵求實驗受試者〗〖實驗名稱〗：早期視覺經驗對視聽整合發展的影響〖實驗單位〗...,https://www.facebook.com/groups/NTU.Head/posts...
7,楚湘雨,4 小時,,,,,討論意涵：今天各地的投票狀況如何0.0今（18日）是四大公投投票日，但根據民進黨統計，截至中...,https://www.facebook.com/groups/NTU.Head/posts...
8,楚湘雨,4 小時,,,,,討論意涵：陳翠蓮教授專訪1920年代，臺灣受到世界思潮的影響，社會運動蓬勃發展。其中「婦女解...,https://www.facebook.com/groups/NTU.Head/posts...
9,劉鈺蘋,5 小時,,,,,《 填問卷抽滿滿的咖啡因！！》花個三分鐘，為期末補充滿滿的咖啡因~嗨~們是修商研所 周善瑜教...,https://www.facebook.com/groups/NTU.Head/posts...


In [37]:
# 建立一個清單，含有各個關鍵字的清單
data_list = [data.loc[data[keyword] == "V"] for keyword in keywords]

all_data = pd.concat(data_list)
all_data = all_data.sort_index()    # 這樣就會依照時間排序
all_data

,name,time,免費,便當,剩下,飲料,content,url
10,許鏵方,6小時,,,,V,住在大一女A棟五樓的朋朋可以看一下有沒有收錯衣服嗎><我大概一個禮拜前晾衣服發現有一件跟我一...,https://www.facebook.com/groups/NTU.Head/posts...
